In [1]:
###if multiple build, exit


import yaml
import pandas as pd
import glob
import sys
import os

In [2]:
model_info_file='/Users/irffanalahi/Research/Research_code/support/MGI_automation/After_MGI_RUN/toydata/filelist.txt'


destinationfolder='/Users/irffanalahi/Research/Research_code/support/MGI_automation/After_MGI_RUN/toydata/destination'
destinationfolder_bam=destinationfolder+"_bam"
destinationfolder_commands=destinationfolder+"_commands"

os.mkdir(destinationfolder)
os.mkdir(destinationfolder_bam)
os.mkdir(destinationfolder_commands)




gmspath='/storage1/fs1/aadel/Active/gmsroot'

model_info_df=pd.read_csv(model_info_file,sep='\t',header=None)

model_info_list=model_info_df[0].tolist()
model_info_list

['bad7948630624c169812530852219653', 'bad7948630624c169812530852262456']

In [3]:

copycommands=[]
cramtobamcommands=[]

for onemodel in model_info_list:
    build=glob.glob(gmspath+"/"+onemodel+'/build*')
    if len(build)!=1:
        print(onemodel)
        print('buld not one. EXITING')
    
    build=build[0]
    
    with open(build+'/inputs.yaml', 'r') as file:
        yamlinfo = yaml.safe_load(file)
        samplename=yamlinfo['sample_name']
    
    copycommand="bsub -M 48000000 -R 'select[mem>48000] rusage[mem=48000]' -oo "+destinationfolder_commands+"/"+samplename+"_copy.log" + "  -q general -a 'docker(irfanwustl/pythonsamtoolsfresh)'  cp -r "+gmspath+"/"+onemodel+' '+destinationfolder+"/"+samplename+"\n"
    
    
    
    copycommands.append(copycommand)
    
    
    cramtobamcommand="bsub -M 48000000 -R 'select[mem>48000] rusage[mem=48000]' -oo "+destinationfolder_commands+"/"+samplename+"_cramtobam.log" + "  -q general -a 'docker(irfanwustl/pythonsamtoolsfresh)'"+ " samtools view -b -T /storage1/fs1/aadel/Active/work/Hg38Reference/all_sequences.fa -o "+destinationfolder_bam+"/"+samplename+".final.bam  "+gmspath+"/"+onemodel+"/build*/results/final.cram\n"

    cramtobamcommands.append(cramtobamcommand)

In [4]:
with open(destinationfolder_commands+"/copy_commands.sh", 'w') as f:
    for item in copycommands:
        f.write("%s" % item)

In [5]:
with open(destinationfolder_commands+"/cramtobam_commands.sh", 'w') as f:
    for item in cramtobamcommands:
        f.write("%s" % item)